In [6]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

if repo_root not in sys.path:
    sys.path.append(repo_root)  # go to parent dir

In [7]:
import yaml
import numpy as np
import matplotlib.pyplot as plt

from spf.rf import beamformer_given_steering_nomean, get_avg_phase

from spf.dataset.spf_dataset import pi_norm

from spf.dataset.v5_data import v5rx_2xf64_keys, v5rx_f64_keys, v5rx_new_dataset

from spf.data_collector import rx_config_from_receiver_yaml
from spf.rf import precompute_steering_vectors
from spf.utils import random_signal_matrix
from spf.sdrpluto.sdr_controller import rx_config_from_receiver_yaml

# V5 data format
from spf.dataset.v5_data import v5rx_new_dataset

import numpy as np
from spf.rf import pi_norm, speed_of_light

"""
theta is the angle from array normal to incident
phi is phase difference

delta_distance  = d*sin(theta)
phi = delta_distance * 2pi / lambda = sin(theta)*d*2pi/lambda
theta = arcsin(lambda * phi / (d*2pi))
"""

fake_yaml = """
# The ip of the emitter
# When the emitter is brought online it is verified
# by a receiver that it actually is broadcasting
emitter:
  type: esp32
  motor_channel: 1

# Two receivers each with two antennas
# When a receiver is brought online it performs
# phase calibration using an emitter equidistant from
# both receiver antenna
# The orientation of the receiver is described in 
# multiples of pi
receivers:
  - receiver-uri: fake
    theta-in-pis: 0
    antenna-spacing-m: 0.05075 # 50.75 mm 
    nelements: 2
    array-type: linear
    rx-gain-mode: fast_attack
    rx-buffers: 2
    rx-gain: -3
    buffer-size: 524288
    f-intermediate: 100000 #1.0e5
    f-carrier: 2412000000 #2.5e9
    f-sampling: 16000000 # 16.0e6
    bandwidth: 300000 #3.0e5
    motor_channel: 0
  - receiver-uri: fake
    theta-in-pis: 0
    antenna-spacing-m: 0.05075 # 50.75 mm 
    nelements: 2
    array-type: linear
    rx-gain-mode: fast_attack
    rx-buffers: 2
    rx-gain: -3
    buffer-size: 524288
    f-intermediate: 100000 #1.0e5
    f-carrier: 2412000000 #2.5e9
    f-sampling: 16000000 # 16.0e6
    bandwidth: 300000 #3.0e5
    motor_channel: 0


n-thetas: 65
n-records-per-receiver: 50
width: 4000
calibration-frames: 10
routine: null
skip_phase_calibration: true
  

data-version: 5
seconds-per-sample: 5.0
"""


def create_fake_dataset(yaml_config_str, filename, orbits=2):
    yaml_config = yaml.safe_load(yaml_config_str)

    with open(f"{filename}.yaml", "w") as outfile:
        yaml.dump(yaml_config, outfile, default_flow_style=False)
    rx_config = rx_config_from_receiver_yaml(yaml_config["receivers"][0])

    _lambda = speed_of_light / rx_config.lo

    m = v5rx_new_dataset(
        filename=f"{filename}.zarr",
        timesteps=yaml_config["n-records-per-receiver"],
        buffer_size=rx_config.buffer_size,
        n_receivers=len(yaml_config["receivers"]),
        chunk_size=512,
        compressor=None,
    )

    thetas = pi_norm(
        np.linspace(0, 2 * np.pi * orbits, yaml_config["n-records-per-receiver"])
    )

    def theta_to_phi(theta, antenna_spacing_m, _lambda):
        return np.sin(theta) * antenna_spacing_m * 2 * np.pi / _lambda

    def phi_to_theta(phi, antenna_spacing_m, _lambda, limit=False):
        sin_arg = _lambda * phi / (antenna_spacing_m * 2 * np.pi)
        # assert sin_arg.min()>-1
        # assert sin_arg.max()<1
        if limit:
            edge = 1 - 1e-8
            sin_arg = np.clip(sin_arg, a_min=-edge, a_max=edge)
        v = np.arcsin(_lambda * phi / (antenna_spacing_m * 2 * np.pi))
        return v, np.pi - v

    rnd_noise = np.random.randn(thetas.shape[0])

    noise = 0.01
    phis_nonoise = theta_to_phi(thetas, rx_config.rx_spacing, _lambda)
    phis = phis_nonoise + rnd_noise * noise
    phis = pi_norm(phis)
    _thetas = phi_to_theta(phis, rx_config.rx_spacing, _lambda, limit=True)

    # signal_matrix = np.vstack([np.exp(1j * phis), np.ones(phis.shape)])

    for receiver_idx in range(2):
        for record_idx in range(yaml_config["n-records-per-receiver"]):
            big_phi = phis_nonoise[[record_idx], None].repeat(
                rx_config.buffer_size, axis=1
            )
            big_phi_with_noise = big_phi + np.random.randn(*big_phi.shape) * noise
            offsets = np.random.uniform(-np.pi, np.pi, big_phi.shape) * 0
            signal_matrix = (
                np.vstack(
                    [np.exp(1j * (offsets + big_phi_with_noise)), np.exp(1j * offsets)]
                )
                * 200
            )
            noise_matrix = random_signal_matrix(
                signal_matrix.reshape(-1).shape[0]
            ).reshape(signal_matrix.shape)
            # add stripes
            window_size = 2048 * 4
            for x in range(0, rx_config.buffer_size, window_size):
                if (x // window_size) % 3 == 0:
                    signal_matrix[:, x : x + window_size] = noise_matrix[
                        :, x : x + window_size
                    ]

            data = {
                "tx_pos_x_mm": np.sin(thetas[record_idx]),
                "tx_pos_y_mm": np.cos(thetas[record_idx]),
                "rx_pos_x_mm": 0,
                "rx_pos_y_mm": 0,
                "system_timestamp": record_idx * 5.0,
                "rx_theta_in_pis": 0,
                "rx_spacing": rx_config.rx_spacing,
                "rx_lo": rx_config.lo,
                "rx_bandwidth": rx_config.rf_bandwidth,
                "avg_phase_diff": get_avg_phase(signal_matrix),
                "rssis": [0, 0],
                "gains": [0, 0],
            }

            z = m[f"receivers/r{receiver_idx}"]
            z.signal_matrix[record_idx] = signal_matrix
            for k in v5rx_f64_keys + v5rx_2xf64_keys:
                z[k][record_idx] = data[k]
            # nthetas = 64 + 1

            # steering_vectors = precompute_steering_vectors(
            #     receiver_positions=rx_config.rx_pos,
            #     carrier_frequency=rx_config.lo,
            #     spacing=nthetas,
            # )
            # beam_sds = beamformer_given_steering_nomean(
            #     steering_vectors=steering_vectors,
            #     signal_matrix=signal_matrix,
            # )


create_fake_dataset(fake_yaml, "test_circle")

/var/folders/31/j84vsqkx1rg7zggf_0glkq8m0000gn/T/ipykernel_22709/272067098.py:41: RuntimeWarning: invalid value encountered in arcsin
  v = np.arcsin(_lambda * phi / (antenna_spacing_m * 2 * np.pi))


In [12]:
signal_matrix.shape

(2, 524288)

In [ ]:
m["receivers/r0"].signal_matrix.shape

(15, 2, 524288)

In [15]:
from spf.dataset.fake_dataset import create_fake_dataset

create_fake_dataset(fake_yaml, "test_circle")

/Users/miskodzamba/Dropbox/research/gits/spf/spf/dataset/fake_dataset.py:119: RuntimeWarning: invalid value encountered in arcsin
  v = np.arcsin(_lambda * phi / (antenna_spacing_m * 2 * np.pi))


In [17]:

    def write_to_record_matrix(self, thread_idx, record_idx, data):
        tx_pos = self.position_controller.controller.position["xy"][
            self.yaml_config["emitter"]["motor_channel"]
        ]
        rx_pos = self.position_controller.controller.position["xy"][
            self.rx_configs[0].motor_channel
        ]

        data.tx_pos_x_mm = tx_pos[0]
        data.tx_pos_y_mm = tx_pos[1]
        data.rx_pos_x_mm = rx_pos[0]
        data.rx_pos_y_mm = rx_pos[1]

        z = self.zarr[f"receivers/r{thread_idx}"]
        z.signal_matrix[record_idx] = data.signal_matrix
        for k in v5rx_f64_keys + v5rx_2xf64_keys:
            z[k][record_idx] = getattr(data, k)

    def close(self):
        self.zarr.store.close()
        self.zarr = None
        zarr_shrink(self.data_filename)